<a href="https://colab.research.google.com/github/owenustc/AToM-OpenMM/blob/master/%E2%80%9C%E2%80%9CBAT_Colab_py_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BAT.py on Colab.

Commands near a ⚠️ sign can only be run in colab's terminal.


In [ ]:
#@title ### Import Drive
from google.colab import drive

drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
#@title Install packages
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh /usr/
! chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local 2>&1 1>/dev/null

#@title Install 6 packages with Conda
!conda install -q -y -c openbabel openbabel --yes 2>&1 1>/dev/null
#@title Install Ambertools
!conda install -c conda-forge ambertools --yes 2>&1 1>/dev/null
#@title Install vmd
!conda install -c conda-forge vmd --yes 2>&1 1>/dev/null
#@title Install Openmmtools
!conda install -c conda-forge openmmtools --yes 2>&1 1>/dev/null
#@title Install openmm
!conda install -c conda-forge openmm --yes 2>&1 1>/dev/null

In [ ]:
#@title Compile MUSTANG
!wget https://lcb.infotech.monash.edu/mustang/mustang_v3.2.3.tgz 2>&1 1>/dev/null
!tar -xvf mustang_v3.2.3.tgz 2>&1 1>/dev/null
%cd /content/MUSTANG_v3.2.3
#@title Install MUSTANG
!make 2>&1 1>/dev/null

#@title Test MUSTANG
!cp /content/MUSTANG_v3.2.3/bin/mustang-3.2.3  /usr/local/bin/
!pwd
!mustang-3.2.3 -f MUSTANG_v3.2.3/data/test/test_zf-CCHH

In [ ]:
#@title Working path
working_path = "/content/drive/MyDrive" #@param {type:"string"}
%cd '{working_path}'


In [ ]:
#@title Download BAT.py
!git clone https://github.com/GHeinzelmann/BAT.py.git

## Stage 1: Equilbration

In [ ]:
#Build equil folder
!cd BAT.py/BAT/
!source ~/.bashrc && cd BAT.py/BAT && python BAT.py -i input-dd-openmm.in -s equil


In [ ]:
%cd /content/drive/MyDrive/BAT.py/BAT/equil/pose0
!sed -i 's/15000000/500000/' equil-08.py 

In [ ]:
#Go to Pose0
!ls /content/drive/MyDrive/BAT.py/BAT/equil/run_files
!cp /content/drive/MyDrive/BAT.py/BAT/equil/run_files/local-equil-op.bash /content/drive/MyDrive/BAT.py/BAT/equil/pose0/ 
%cd /content/drive/MyDrive/BAT.py/BAT/equil/pose0/ 
!cat local-equil-op.bash
#Replace RANGE with '8'
!sed -i 's/RANGE/8/' local-equil-op.bash && cat local-equil-op.bash
##Start simulation
!bash local-equil-op.bash


In [ ]:
##Free energy
%cd /content/drive/MyDrive/BAT.py/BAT/ 
!ls && python BAT.py -i input-dd-openmm.in -s fe
%cd /content/drive/MyDrive/BAT.py/BAT/fe/pose0 
!cp /content/drive/MyDrive/BAT.py/BAT/run_files/run-op-dd.bash ./
#@title Modify run script
%cd fe/pose0
!cp ../../run_files/run-op-dd.bash .
!sed -i 's/qsub PBS-a/bash local-rest.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-l/bash local-rest.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-t/bash local-rest.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-c/bash local-rest.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-r/bash local-rest.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-e/bash local-dd.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-v/bash local-dd.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-w/bash local-dd.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-w/bash local-dd.bash/g ' run-op-dd.bash
!sed -i 's/qsub PBS-f/bash local-dd.bash/g ' run-op-dd.bash
!bash run-op-dd.bash



In [ ]:
%cd /content/drive/MyDrive/BAT.py/BAT/fe/pose0 
!bash run-op-dd.bash 

## Analysis 

 Run in terminal⚠️
```
%cd ../../
!python BAT.py -i input-dd.in -s analysis
```

In [ ]:
#@title Job control
while True:pass